In [1]:
from arcgis.gis import GIS
import arcpy
from datetime import date
import timeit
import pandas as pd
from itertools import groupby
from operator import itemgetter

In [2]:
gis = GIS('home')

In [3]:
search_result = gis.content.search(query="title:FERNSSummary_McKenzie_Joined", item_type="Feature Layer")

In [4]:
item = gis.content.get('5a6cd762152a48cc9be95b9ccf6272bd')

In [5]:
lyr = [lyr.url for lyr in item.layers if lyr.properties.name == 'FERNSSummary_McKenzie_Joined'][0]
print(lyr)

https://services5.arcgis.com/9s1YtFmLS0YTl10F/arcgis/rest/services/FERNS_for_McKenzie_Catchments/FeatureServer/0


In [6]:
path = r"G:\projects\UtilityDistricts\eweb\DrinkingWater\EPA319_NPS_grant\ForestApplication\ODF_FACTS_DB\FERNS\FERNS_Model"
wrkspace = path + "\\FERNS_Workspace.gdb"

In [7]:
year = str(date.today().year)

In [8]:
Expression="YEAR = '{0}'".format(year)

In [9]:
Expression

"YEAR = '2023'"

In [10]:
desc = arcpy.Describe(lyr)

In [11]:
desc

catalogPath,https://services5.arcgis.com/9s1YtFmLS0YTl10F/arcgis/rest/services/FERNS_for_McKenzie_Catchments/FeatureServer/0
dataType,FeatureClass
shapeType,Polygon
hasM,False
hasZ,False
OBJECTID1,OID
Basin_Name,String
EWEB_ID,String
Subbasin_Acres,Double
YEAR,String
ActType,String


In [12]:
def unique_values(table, field):  ##uses list comprehension
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

In [13]:
try:
    start = timeit.default_timer()
    vals = unique_values(lyr,"YEAR")

    stop = timeit.default_timer()
    total_time = stop - start
    print(total_time)
    print(vals)

except Exception as e:
    print("Error: " + e.args[0])

0.8151014000000032
['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']


In [14]:
fields = arcpy.ListFields(lyr)

for field in fields:
    print(f"{field.name} is a type of {field.type} with a length of {field.length}")

OBJECTID1 is a type of OID with a length of 0
Basin_Name is a type of String with a length of 10485758
EWEB_ID is a type of String with a length of 10485758
Subbasin_Acres is a type of Double with a length of 0
YEAR is a type of String with a length of 10485758
ActType is a type of String with a length of 10485758
FREQUENCY is a type of Integer with a length of 0
SUM_calc_acres is a type of Double with a length of 0
Activity_Acres_Percentage is a type of Double with a length of 0
Shape__Area is a type of Double with a length of 0
Shape__Length is a type of Double with a length of 0
Shape is a type of Geometry with a length of 0


In [15]:
field_names = [field.name for field in fields]

In [16]:
field_names.remove('OBJECTID1')

In [17]:
field_names

['Basin_Name',
 'EWEB_ID',
 'Subbasin_Acres',
 'YEAR',
 'ActType',
 'FREQUENCY',
 'SUM_calc_acres',
 'Activity_Acres_Percentage',
 'Shape__Area',
 'Shape__Length',
 'Shape']

In [18]:
AGO_FERNSSummary_McKenzie_Joined_2_, Count = arcpy.management.SelectLayerByAttribute(in_layer_or_view=lyr, 
                                                                                        selection_type="NEW_SELECTION", 
                                                                                        where_clause=Expression, 
                                                                                        invert_where_clause="")

In [19]:
print(f"Year {year} has {Count} count!")

Year 2023 has 36 count!


In [22]:
arcpy.env.overwriteOutput = True

In [60]:
start = timeit.default_timer()
result = arcpy.management.FindIdentical(lyr, path+'\\duplicate_incidents.dbf', field_names)
stop = timeit.default_timer()
total_time = stop - start
print(total_time)

329.43447989999913


In [61]:
out_records = []   
for row in arcpy.SearchCursor(result.getOutput(0), fields="IN_FID; FEAT_SEQ"):
    out_records.append([row.IN_FID, row.FEAT_SEQ])

In [62]:
out_records.sort(key = itemgetter(1))

In [63]:
identicals_iter = groupby(out_records, itemgetter(1))

In [64]:
identical_groups = [[item[0] for item in data] for (key, data) in identicals_iter]

In [65]:
len([item for item in identical_groups if len(item) == 1])

1369

In [66]:
len(identical_groups)

1387

In [67]:
[item for item in identical_groups if len(item) > 1]

[[4738, 4756],
 [4739, 4757],
 [4740, 4758],
 [4741, 4759],
 [4742, 4760],
 [4743, 4761],
 [4744, 4762],
 [4745, 4763],
 [4746, 4764],
 [4747, 4765],
 [4748, 4766],
 [4749, 4767],
 [4750, 4768],
 [4751, 4769],
 [4752, 4770],
 [4753, 4771],
 [4754, 4772],
 [4755, 4773]]

In [68]:
start = timeit.default_timer()
res = arcpy.management.DeleteIdentical(lyr, field_names)
stop = timeit.default_timer()
total_time = stop - start
print(total_time)

ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
ERROR: code:504, Your request has timed out., Proxy server got bad address from remote server (verify the server is running).
Failed to execute (DeleteIdentical).


In [46]:
year

'2023'

In [51]:
Expression=f"YEAR <> '{year}'"

In [43]:
lyr

'https://services5.arcgis.com/9s1YtFmLS0YTl10F/arcgis/rest/services/FERNS_for_McKenzie_Catchments/FeatureServer/0'

In [52]:
selected, count = arcpy.management.SelectLayerByAttribute(in_layer_or_view=lyr, 
                                                          selection_type="NEW_SELECTION",
                                                          where_clause=Expression, 
                                                          invert_where_clause="")

In [54]:
FERNSSummary_McKenzie_Joined = path + "\\FERNS_Final_Products.gdb\\FERNSSummary_McKenzie_Joined"

In [55]:
Joined = arcpy.management.Append(inputs=[FERNSSummary_McKenzie_Joined],
                                 target=selected,
                                 schema_type="TEST", 
                                 field_mapping="", 
                                 subtype="", 
                                 expression="")[0]